## Load Data from CSVs

In [1]:
import unicodecsv

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')

In [2]:
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', '2014-11-10'),
             ('cancel_date', '2015-01-14'),
             ('days_to_cancel', '65'),
             ('is_udacity', 'True'),
             ('is_canceled', 'True')])

In [3]:
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', '2015-01-09'),
             ('num_courses_visited', '1.0'),
             ('total_minutes_visited', '11.6793745'),
             ('lessons_completed', '0.0'),
             ('projects_completed', '0.0')])

In [4]:
project_submissions[0]

OrderedDict([('creation_date', '2015-01-14'),
             ('completion_date', '2015-01-16'),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

## Fixing Data Types

In [5]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2014, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('days_to_cancel', 65),
             ('is_udacity', True),
             ('is_canceled', True)])

In [6]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0)])

In [7]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

## Investigating the Data and Problems in the Data

In [8]:
# Change the "acct" field to "account_key" in the daily_engagement to be consistent with the other tables
for engagement_record in daily_engagement:
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]

In [9]:
# Returns a set with the unique values of "account_key" within the given data
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

In [10]:
# See how many enrollments there are
len(enrollments)

1640

In [11]:
# See how many unique students enrolled
unique_enrolled_students = get_unique_students(enrollments)
len(unique_enrolled_students)

1302

In [12]:
# See how many records of daily engagement we have
len(daily_engagement)

136240

In [13]:
# See how many unique students we have engagement data on
unique_engagement_students = get_unique_students(daily_engagement)
len(unique_engagement_students)

1237

In [14]:
# See how many projects were submitted
len(project_submissions)

3642

In [15]:
# See how many unique students submitted projects
unique_project_submitters = get_unique_students(project_submissions)
len(unique_project_submitters)

743

## Missing Engagement Records

In [16]:
# Look at a student who enrolled but doesn't have engagement numbers
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students:
        print(enrollment)
        break

OrderedDict([('account_key', '1219'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 12, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)])


## Checking for More Problem Records

In [17]:
# Find students who enrolled, don't have engagement numbers, and stayed enrolled at least a day
num_problem_students = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students and enrollment['join_date'] != enrollment['cancel_date']:
        print(enrollment)
        num_problem_students += 1

num_problem_students

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])


3

## Tracking Down the Remaining Problems

In [18]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [19]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [20]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print (len(non_udacity_submissions))

1622
135656
3634


## Refining the Question

In [21]:
# Create a dictionary mapping students who did not cancel within 7 days to their most recent
# join date
paid_students = {}
for enrollment in non_udacity_enrollments:
    # If the student is still enrolled or they took more than 7 days to cancel, we will consider them
    # a paid student
    if not enrollment['is_canceled'] or enrollment['days_to_cancel'] > 7:
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        
        # Add this student to the dictionary if they are not already present, or if this enrollment date
        # is the most recent one we've seen so far
        if account_key not in paid_students or enrollment_date > paid_students[account_key]:
            paid_students[account_key] = enrollment_date

len(paid_students)

995

## Getting Data from First Week

In [22]:
# Given some data with an account_key field, removes any records corresponding to accounts that canceled with
# seven days
def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

In [23]:
# Remove data from students who canceled within 7 days from all three tables
paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print (len(paid_enrollments))
print (len(paid_engagement))
print (len(paid_submissions))

1293
134549
3618


In [24]:
# Create a binary version of the num_courses_visited field, equal to
# 1 if the student visited at least one course, and 0 otherwise.
for engagement_record in paid_engagement:
    if engagement_record['num_courses_visited'] > 0:
        engagement_record['has_visited'] = 1
    else:
        engagement_record['has_visited'] = 0

In [25]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days >= 0 and time_delta.days < 7

# Collect all the engagement records that come in the first week of the student's most recent enrollment
paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']
    
    if within_one_week(join_date, engagement_record_date):
        paid_engagement_in_first_week.append(engagement_record)

len(paid_engagement_in_first_week)

6919

## Exploring Student Engagement

In [26]:
from collections import defaultdict

# Create a dictionary of engagement grouped by the given key name.
# The keys are the fields from the data in the key_name column,
# and the values are lists of data points.
def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

engagement_by_account = group_data(paid_engagement_in_first_week, 'account_key')

In [27]:
# Takes a dictionary of grouped data as output by group_data, as well as a field name to sum over,
# and returns a new dictionary with that field summed up and other fields discarded.
# The keys of the resulting dictionary will be the same as the keys of the original,
# and the values will be numbers containing the total value of the given field across all data points for the
# associated key.
def sum_grouped_items(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data

total_minutes_by_account = sum_grouped_items(engagement_by_account, 'total_minutes_visited')

In [38]:
# Show matplotlib plots within the notebook rather than in a new window.
# Omit this line if not using IPython notebook
%pylab inline

import matplotlib.pyplot as plt
import numpy as np

# Summarize the given data
def describe_data(data):
    print (np.mean(data))
    print (np.std(data))
    print(np.min(data))
    print(np.max(data))
    print(np.mean(data))

    
describe_data(total_minutes_by_account.values())

Populating the interactive namespace from numpy and matplotlib
dict_values([494.88049616599994, 18.576384666670002, 0.0, 0.0, 33.3214046667, 329.7578566663, 780.4545511666701, 104.20388850009999, 989.113641833, 448.471384167, 130.12347833367, 0.0, 179.4719088333, 1013.3833969996999, 65.6221875, 983.375040335, 119.12030049999998, 235.49969150033, 155.1361575, 0.0, 447.93897783336996, 931.1036911666699, 657.2052335000001, 1591.3228143334, 943.188117167, 0.0, 430.801675833, 1579.12122666663, 766.256315667, 556.1906033333, 69.6578351667, 0.0, 123.2915048333, 253.9870258334, 180.413814, 765.6402170004, 809.2138958339, 1378.195091668, 0.0, 1001.5888595, 478.21952616690004, 511.925391, 576.4643026663, 88.822038, 0.0, 0.0, 26.1189351667, 854.8288881656, 1055.6871896667, 0.0, 491.9087078335, 0.0, 3.22752383333, 401.97479050029995, 427.764673834, 589.6171318323301, 106.72208133333001, 91.3514995, 1024.109326834, 47.1679638333, 0.0, 496.99219366667, 293.383122167, 465.58233333400005, 346.50302683

TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'

## Debugging Data Analysis Code

In [39]:
# Find the student who spent the most minutes during the first week

student_with_max_minutes = None
max_minutes = 0

for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_with_max_minutes = student
        
max_minutes

3564.7332644989997

In [40]:
# Print out every engagement record for the student who spent the most minutes
for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] == student_with_max_minutes:
        print (engagement_record)

OrderedDict([('utc_date', datetime.datetime(2015, 7, 9, 0, 0)), ('num_courses_visited', 4), ('total_minutes_visited', 850.519339666), ('lessons_completed', 4), ('projects_completed', 0), ('account_key', '163'), ('has_visited', 1)])
OrderedDict([('utc_date', datetime.datetime(2015, 7, 10, 0, 0)), ('num_courses_visited', 6), ('total_minutes_visited', 872.633923334), ('lessons_completed', 6), ('projects_completed', 0), ('account_key', '163'), ('has_visited', 1)])
OrderedDict([('utc_date', datetime.datetime(2015, 7, 11, 0, 0)), ('num_courses_visited', 2), ('total_minutes_visited', 777.018903666), ('lessons_completed', 6), ('projects_completed', 0), ('account_key', '163'), ('has_visited', 1)])
OrderedDict([('utc_date', datetime.datetime(2015, 7, 12, 0, 0)), ('num_courses_visited', 1), ('total_minutes_visited', 294.568774), ('lessons_completed', 2), ('projects_completed', 0), ('account_key', '163'), ('has_visited', 1)])
OrderedDict([('utc_date', datetime.datetime(2015, 7, 13, 0, 0)), ('num_c

## Lessons Completed in First Week

In [41]:
lessons_completed_by_account = sum_grouped_items(engagement_by_account, 'lessons_completed')
describe_data(lessons_completed_by_account.values())

dict_values([1, 0, 0, 0, 0, 0, 6, 1, 7, 2, 0, 0, 0, 10, 0, 1, 1, 1, 0, 0, 2, 12, 5, 6, 4, 0, 2, 3, 2, 5, 1, 0, 0, 0, 1, 4, 4, 5, 0, 10, 6, 1, 2, 0, 0, 0, 0, 13, 12, 0, 2, 0, 0, 3, 1, 4, 0, 1, 3, 0, 0, 2, 1, 5, 6, 3, 0, 2, 0, 0, 2, 0, 7, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 3, 5, 1, 0, 1, 0, 1, 1, 1, 3, 3, 0, 0, 0, 2, 2, 1, 0, 0, 7, 2, 0, 0, 1, 0, 3, 9, 1, 2, 0, 1, 0, 5, 1, 5, 0, 3, 3, 7, 0, 0, 9, 0, 1, 5, 1, 0, 0, 1, 0, 6, 2, 0, 3, 0, 1, 3, 2, 1, 1, 1, 4, 0, 0, 0, 0, 1, 7, 4, 1, 4, 0, 2, 0, 0, 0, 2, 3, 3, 20, 15, 2, 0, 0, 0, 0, 2, 5, 4, 0, 1, 10, 6, 3, 6, 5, 0, 3, 1, 2, 2, 0, 2, 2, 0, 7, 0, 0, 4, 0, 1, 4, 1, 9, 1, 6, 1, 0, 0, 2, 2, 1, 3, 0, 1, 0, 1, 5, 0, 6, 0, 1, 1, 0, 20, 2, 4, 1, 1, 0, 0, 1, 3, 1, 3, 0, 0, 0, 5, 1, 4, 0, 3, 2, 2, 0, 0, 1, 3, 0, 0, 2, 1, 1, 5, 2, 1, 2, 0, 0, 1, 0, 0, 3, 3, 5, 0, 0, 2, 1, 3, 10, 1, 5, 1, 9, 5, 8, 0, 6, 0, 3, 2, 0, 0, 1, 0, 2, 0, 1, 0, 2, 2, 0, 7, 3, 0, 0, 4, 2, 2, 0, 2, 1, 0, 0, 6, 2, 4, 1, 3, 0, 0, 2, 0, 9, 0, 5, 9, 3, 9, 17, 0, 5, 3, 0, 0, 2, 6, 1, 7, 1, 2

TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'

## Number of Visits in First Week

In [42]:
days_visited_by_account = sum_grouped_items(engagement_by_account, 'has_visited')
describe_data(days_visited_by_account.values())

dict_values([7, 2, 0, 0, 1, 5, 6, 3, 7, 4, 3, 0, 2, 6, 1, 5, 2, 2, 4, 0, 5, 7, 5, 7, 3, 0, 3, 5, 4, 5, 2, 0, 2, 5, 1, 6, 7, 7, 0, 4, 5, 4, 6, 2, 0, 0, 1, 6, 7, 0, 7, 0, 1, 3, 5, 4, 4, 3, 7, 1, 0, 6, 2, 5, 4, 6, 1, 6, 0, 0, 3, 3, 6, 0, 3, 1, 4, 4, 0, 3, 2, 3, 6, 4, 6, 3, 0, 4, 0, 3, 3, 2, 3, 5, 0, 1, 1, 5, 4, 3, 4, 2, 6, 6, 2, 1, 3, 4, 6, 5, 1, 6, 2, 4, 3, 7, 3, 7, 5, 3, 6, 5, 0, 1, 7, 1, 3, 3, 1, 0, 0, 6, 3, 7, 5, 1, 3, 1, 1, 7, 6, 2, 3, 5, 5, 3, 0, 0, 1, 4, 7, 5, 1, 6, 0, 6, 1, 0, 0, 7, 6, 6, 6, 6, 6, 0, 0, 2, 1, 5, 7, 6, 1, 6, 7, 5, 5, 6, 7, 2, 5, 3, 4, 4, 0, 4, 7, 1, 5, 1, 4, 5, 4, 6, 6, 6, 7, 6, 6, 7, 0, 3, 7, 7, 4, 7, 5, 4, 1, 3, 6, 3, 4, 1, 5, 3, 0, 7, 5, 4, 3, 3, 3, 2, 7, 6, 4, 5, 2, 5, 1, 7, 2, 5, 0, 6, 5, 4, 1, 0, 4, 7, 2, 3, 4, 6, 1, 4, 2, 3, 6, 0, 6, 4, 0, 3, 6, 5, 5, 0, 1, 5, 5, 3, 6, 3, 4, 2, 6, 6, 7, 3, 7, 2, 5, 7, 0, 1, 5, 0, 3, 1, 1, 1, 2, 5, 1, 6, 5, 0, 3, 5, 4, 5, 3, 3, 3, 0, 2, 5, 3, 7, 2, 7, 0, 1, 5, 4, 5, 2, 7, 7, 5, 7, 7, 3, 3, 5, 0, 0, 4, 7, 4, 6, 3, 5, 5, 4, 3, 

TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'

## Splitting out Passing Students

In [43]:
# Create a set of all account keys that eventually passed the subway project

subway_project_lesson_keys = ['746169184', '3176718735']

pass_subway_project = set()

for submission in paid_submissions:
    project = submission['lesson_key']
    rating = submission['assigned_rating']
    
    # These two project ids correspond to different versions of the subway project, and both
    # PASSED and DISTINCTION indicate that the student passed the project
    if (project in subway_project_lesson_keys) and (rating == 'PASSED' or rating == 'DISTINCTION'):
        pass_subway_project.add(submission['account_key'])
        
len(pass_subway_project)

647

In [44]:
# Engagement data for students who eventually pass the subway project
passing_engagement = []
# Engagement data for students who do not eventually pass the subway project
non_passing_engagement = []

for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] in pass_subway_project:
        passing_engagement.append(engagement_record)
    else:
        non_passing_engagement.append(engagement_record)

## Comparing the Two Student Groups

In [45]:
passing_engagement_by_account = group_data(passing_engagement, 'account_key')
non_passing_engagement_by_account = group_data(non_passing_engagement, 'account_key')

In [46]:
print ('non-passing students:')
non_passing_minutes = sum_grouped_items(non_passing_engagement_by_account, 'total_minutes_visited')
describe_data(non_passing_minutes.values())

# Adding plt.show() after each plot will cause the two plots in this cell to display separately,
# rather than on top of each other
plt.show()

print ('passing students:')
passing_minutes = sum_grouped_items(passing_engagement_by_account, 'total_minutes_visited')
describe_data(passing_minutes.values())
plt.show()

non-passing students:
dict_values([736.4371648339999, 177.8367486667, 0.0, 0.0, 193.848226, 92.5547231666, 0.0, 138.052647, 144.1243173333, 0.0, 454.273163499, 115.15559099992998, 32.816891, 348.39415716703, 171.593366833, 16.5745846667, 100.0702155, 25.243540000000003, 9.61211033333, 18.889756499999997, 0.0, 0.0, 16.604203, 0.0, 198.6962631669, 666.3290910003, 88.1119545, 56.45108966666999, 0.0, 0.0, 0.0, 1329.9598305, 88.0103965, 3.56199133333, 253.23675750029997, 139.060529, 307.3006509996, 0.0, 9.4907925, 401.81893949967, 29.9067615, 21.1284496667, 0.0, 41.7199715, 219.926066667, 0.0, 313.50181033333, 489.7736965, 0.0, 374.5761038331, 706.5435295006, 0.0, 324.6302378336, 52.1289673334, 30.5951761667, 116.628876833, 413.3262124997, 34.0330873333, 0.0, 354.6223496663, 0.0, 100.06368333333, 85.29257, 59.340783666700005, 135.6825435, 275.16640700005996, 0.0, 0.0, 199.31994916669998, 48.1974151667, 4.2604045, 0.0, 0.0, 277.8489021664, 176.3818004999, 4.78649716667, 23.695871500029998, 0

TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'

In [47]:
print 'non-passing students:'
non_passing_lessons = sum_grouped_items(non_passing_engagement_by_account, 'lessons_completed')
describe_data(non_passing_lessons.values())
plt.show()

print 'passing students:'
passing_lessons = sum_grouped_items(passing_engagement_by_account, 'lessons_completed')
describe_data(passing_lessons.values())
plt.show()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-47-7ee537a04be1>, line 1)

In [48]:
print 'non-passing students:'
non_passing_visits = sum_grouped_items(non_passing_engagement_by_account, 'has_visited')
describe_data(non_passing_visits.values())
plt.show()

print 'passing students:'
passing_visits = sum_grouped_items(passing_engagement_by_account, 'has_visited')
describe_data(passing_visits.values())
plt.show()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-48-f712ef146e64>, line 1)

## Improving Plots and Sharing Findings

In [49]:
# Import seaborn for aesthetic improvements
import seaborn as sns

# Use bins=8 since there are exactly 8 possible values for this variable - 0 through 7 inclusive
plt.hist(non_passing_visits.values(), bins=8)
plt.xlabel('Number of days')
plt.title('Distribution of classroom visits in the first week for students who do not pass the subway project')

NameError: name 'non_passing_visits' is not defined

In [50]:
plt.hist(passing_visits.values(), bins=8)
plt.xlabel('Number of days')
plt.title('Distribution of classroom visits in the first week for students who pass the subway project')

NameError: name 'passing_visits' is not defined